<a href="https://colab.research.google.com/github/gerges-marzouk/imaging-project-/blob/main/final_unsharpen_muli_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

gergesmarzouk_barin_tumor_mri_dataset_path = kagglehub.dataset_download('gergesmarzouk/barin-tumor-mri-dataset')

print('Data source import complete.')


# Import the neccessary liberaries

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB1,EfficientNetB2,EfficientNetB3,EfficientNetB4,EfficientNetB5
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint,LearningRateScheduler
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve2d
import scipy.ndimage as ndimage
from skimage import data, io
from matplotlib import pyplot as plt
import cv2
import numpy as np
from skimage import img_as_ubyte,img_as_float
from skimage import data, io, filters
from skimage.filters.rank import median
from skimage.morphology import disk
from matplotlib.pyplot import imshow, show, subplot, title, get_cmap, hist
from scipy import ndimage
from scipy.fftpack import fft , fft2 ,fftshift , ifftshift , ifft2
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# image processing techniques


# 1-Average filter (bluring filter )


In [ ]:
image_1 =cv2.imread("/kaggle/input/barin-tumor-mri-dataset/Germinoma T2/076a6fa8613c4a4f8b05f4383280b9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# tumor image
image_2=cv2.imread("/kaggle/input/barin-tumor-mri-dataset/_NORMAL T2/02ac46b7eab4a327a468b0f26d17c9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# normal image
average_blurred_image_1=cv2.blur(image_1 , (3,3))
average_blurred_image_2=cv2.blur(image_2,(3,3))
plt.figure(figsize=(10,6))
plt.subplot(321)
plt.imshow(image_1 , cmap='gray')
plt.title(" original tumor image ")
plt.axis('off')
plt.subplot(322)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")
plt.axis('off')
plt.subplot(323)
plt.imshow(average_blurred_image_1, cmap='gray')
plt.title("blured tumor image with kernel (3,3) ")
plt.axis('off')
plt.subplot(324)
plt.imshow(average_blurred_image_2, cmap='gray')
plt.title("blured non_tumor image with kernel (3,3) ")
plt.axis('off')
average_blurred_image_11=cv2.blur(image_1 , (7,7))
average_blurred_image_22=cv2.blur(image_2,(7,7))
plt.subplot(325)
plt.imshow(average_blurred_image_11, cmap='gray')
plt.title("blured tumor image with kernel (7,7) ")
plt.axis('off')
plt.subplot(326)
plt.imshow(average_blurred_image_22, cmap='gray')
plt.title("blured non_tumor image with kernel (7,7) ")
plt.axis('off')

plt.show()



# 2-Median filter

In [ ]:
image_1 =cv2.imread("/kaggle/input/barin-tumor-mri-dataset/Germinoma T2/076a6fa8613c4a4f8b05f4383280b9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# tumor image
image_2=cv2.imread("/kaggle/input/barin-tumor-mri-dataset/_NORMAL T2/02ac46b7eab4a327a468b0f26d17c9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# normal image
median_filtered_image_1=cv2.medianBlur(image_1 , 3)
median_filtered_image_2=cv2.medianBlur(image_2,3)
plt.figure(figsize=(10,6))
plt.subplot(321)
plt.imshow(image_1 , cmap='gray')
plt.title(" original tumor image ")
plt.axis('off')
plt.subplot(322)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")
plt.axis('off')
plt.subplot(323)
plt.imshow(median_filtered_image_1, cmap='gray')
plt.title("median filtered  tumor image with kernel (3,3) ")
plt.axis('off')
plt.subplot(324)
plt.imshow(median_filtered_image_2, cmap='gray')
plt.title("median filtered non_tumor image with kernel (3,3) ")
plt.axis('off')
median_filtered_image_11=cv2.medianBlur(image_1 , 7)
median_filtered_image_22=cv2.medianBlur(image_2,7)
plt.subplot(325)
plt.imshow(median_filtered_image_11, cmap='gray')
plt.title("median filtered  tumor image with kernel (7,7) ")
plt.axis('off')
plt.subplot(326)
plt.imshow(median_filtered_image_22, cmap='gray')
plt.title("median filtered  non_tumor image with kernel (7,7) ")
plt.axis('off')

plt.show()

# 3-Gaussian filter

In [ ]:

image_1 =cv2.imread("/kaggle/input/barin-tumor-mri-dataset/Germinoma T2/076a6fa8613c4a4f8b05f4383280b9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# tumor image
image_2=cv2.imread("/kaggle/input/barin-tumor-mri-dataset/_NORMAL T2/02ac46b7eab4a327a468b0f26d17c9_big_gallery.jpeg", cv2.IMREAD_GRAYSCALE)# normal image
plt.figure(figsize=(10,5))
plt.subplot(321)
plt.imshow(image_1, cmap='gray')
plt.title(" original  tumor image")
plt.axis('off')
plt.subplot(322)
plt.imshow(cv2.GaussianBlur(image_1,(3,3),0), cmap='gray')
plt.title(" gaussain filtered tumor image with kernel size = 3")
plt.axis('off')
plt.subplot(323)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")
plt.axis('off')
plt.subplot(324)
plt.imshow(cv2.GaussianBlur(image_2,(3,3),0), cmap='gray')
plt.title(" gaussain filtered non_tumor image with kernel size = 3")
plt.axis('off')
plt.subplot(325)
plt.imshow(cv2.GaussianBlur(image_1,(9,9),0), cmap='gray')
plt.title(" gaussain filtered tumor image with kernel size = 9")
plt.axis('off')
plt.subplot(326)
plt.imshow(cv2.GaussianBlur(image_2,(9,9),0), cmap='gray')
plt.title(" gaussain filtered non_tumor image with kernel size = 3")
plt.axis('off')
plt.show()

# 4-Bilateral filter

In [ ]:
# in this filter the ones that can control in the output image are depth d which control in the diameter of each pixel neighborhood - sigmaColor parameter that control in the distance between the colors .
# sigmaSpace parameter is that one which control in the distance between pixels and each others on the coordinates .
plt.figure(figsize=(10,8))
plt.subplot(221)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")
plt.axis('off')

plt.subplot(222)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")
plt.axis('off')

plt.subplot(223)
plt.imshow(cv2.bilateralFilter(image_1 , d=10 ,sigmaColor=100 , sigmaSpace=100), cmap='gray')
plt.title("Bilateral filtered  tumor image ")
plt.axis('off')

plt.subplot(224)
plt.imshow(cv2.bilateralFilter(image_2 , d=10 ,sigmaColor=100 , sigmaSpace=100), cmap='gray')
plt.title("Bilateral filtered non-tumor image ")
plt.axis('off')
plt.show()

#  5- Outlier method

In [ ]:
outlier_filtering_image_1 = image_1.copy()
k = 3
pad = k // 2
T = 30
padded = np.pad(image_1, pad, mode='reflect')
for i in range(image_1.shape[0]):
    for j in range(image_1.shape[1]):
        window = padded[i:i+k, j:j+k]
        med = np.median(window) # Median Based
        if abs(int(image_1[i, j]) - med) > T:
            outlier_filtering_image_1[i, j] = med

outlier_filtering_image_2 = image_2.copy()
k = 3
pad = k // 2
T = 30
padded = np.pad(image_2, pad, mode='reflect')
for i in range(image_2.shape[0]):
    for j in range(image_2.shape[1]):
        window = padded[i:i+k, j:j+k]
        med = np.median(window) # Median Based
        if abs(int(image_2[i, j]) - med) > T:
            outlier_filtering_image_2[i, j] = med

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(outlier_filtering_image_1 , cmap='gray')
plt.title("outlier filtered tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")

plt.subplot(144)
plt.imshow(outlier_filtering_image_2 , cmap='gray')
plt.title(" outlier filtered non_tumor image ")

plt.show()

# 6- Contrast stretching

In [ ]:
pixels_img_1= image_1[image_1 > 0]
pixels_img_2= image_2[image_2 > 0]

hist_image_1= cv2.calcHist([pixels_img_1] , [0], None,[256], [0,256])
hist_image_2= cv2.calcHist([pixels_img_2] , [0], None,[256], [0,256])
plt.figure(figsize=(18,6))
plt.subplot(221)
plt.imshow(image_1, cmap='gray')
plt.title(" tumor original image ")

plt.subplot(222)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("original tumor  image histogram ")
plt.plot(hist_image_1)
plt.xlim([0,256])

plt.subplot(223)
plt.imshow(image_2, cmap='gray')
plt.title(" non_tumor original image ")

plt.subplot(224)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("original non_tumor  image histogram ")
plt.plot(hist_image_2)
plt.xlim([0,256])
plt.show()

In [ ]:
def contrast_stretching(img, k,s):
    s = np.where(s < k, (s/k)*127, 127 + ((s-k)/(255-k))*128).astype(np.uint8)
    return s

In [ ]:
s = cv2.normalize(image_1, None, 0, 255, cv2.NORM_MINMAX)
k = 100
stretched_image_1 = contrast_stretching(image_1, k,s)

s = cv2.normalize(image_2, None, 0, 255, cv2.NORM_MINMAX)
stretched_image_2 = contrast_stretching(image_2, k,s)

stre_pixels_1= stretched_image_1[stretched_image_1 >0]
stre_pixels_2= stretched_image_2[stretched_image_2 >0]

stret_hist_image_1=  cv2.calcHist([stre_pixels_1] , [0], None,[256], [0,256])
stret_hist_image_2=  cv2.calcHist([stre_pixels_2] , [0], None,[256], [0,256])

In [ ]:

plt.figure(figsize=(18,6))
plt.subplot(141)
plt.imshow(stretched_image_1 , cmap='gray')
plt.title("stretched tumor  image ")

plt.subplot(142)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("stretched tumor  image histogram ")
plt.plot(stret_hist_image_1)
plt.xlim([0,256])

plt.subplot(143)
plt.imshow(stretched_image_2, cmap='gray')
plt.title(" strteched non_tumor  image ")

plt.subplot(144)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("stretched  non_tumor  image histogram ")
plt.plot(stret_hist_image_2)
plt.xlim([0,256])
plt.show()

# 7- Thresholding

In [ ]:
def apply_thresholding(image, k):
    _, thresholded = cv2.threshold(image, k, 255, cv2.THRESH_BINARY)
    return thresholded

In [ ]:
T=75
thresholded_image_1= apply_thresholding(image_1, T)
thresholded_image_2= apply_thresholding(image_2, T)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title(" Tumor image OIG")

plt.subplot(142)
plt.imshow(thresholded_image_1, cmap='gray')
plt.title(" thresholded tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" non_Tumor image OIG")

plt.subplot(144)
plt.imshow(thresholded_image_2, cmap='gray')
plt.title(" thresholded non_tumor image ")
plt.show()

# 8-histogram equalization


In [ ]:
equalized_image_1 = cv2.equalizeHist(image_1)
pixels3 = equalized_image_1[equalized_image_1 > 0]

equalized_image_2 = cv2.equalizeHist(image_2)
pixels4 = equalized_image_2[equalized_image_2 > 0]

In [ ]:
tumor_hist_equalized = cv2.calcHist([pixels3], [0], None, [256], [0, 256])
non_tumor_hist_equalized = cv2.calcHist([pixels4], [0], None, [256], [0, 256])

plt.figure(figsize=(15,12))
plt.subplot(221)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("original tumor  image histogram ")
plt.plot(hist_image_1)
plt.xlim([0,256])

plt.subplot(222)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("equalized  tumor  image histogram ")
plt.plot(tumor_hist_equalized)
plt.xlim([0,256])

plt.subplot(223)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("original non_tumor  image histogram ")
plt.plot(hist_image_2)
plt.xlim([0,256])

plt.subplot(224)
plt.xlabel("gray intensity ")
plt.ylabel("pixels number ")
plt.title("original non_tumor  image histogram ")
plt.plot(non_tumor_hist_equalized)
plt.xlim([0,256])

plt.show()


# 9- Negative transformation

In [ ]:
neg_image_1= 255-image_1
neg_image_2= 255- image_2

plt.figure(figsize=(15,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title(" Tumor image OIG")

plt.subplot(142)
plt.imshow(neg_image_1, cmap='gray')
plt.title(" negative tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" non_Tumor image OIG")

plt.subplot(144)
plt.imshow(neg_image_2, cmap='gray')
plt.title(" negative non_tumor image ")
plt.show()


# 10-Logarithmic transformation function

In [ ]:
def log_transform(image, c=1):
    return c * np.log1p(image)

In [ ]:
c_values = [0.5, 1, 5, 10]

plt.figure(figsize=(10, 5))
for i, c in enumerate(c_values, 1):
    log_image = log_transform(image_1, c)
    plt.subplot(1, len(c_values), i)
    plt.imshow(log_image, cmap='gray')
    plt.title(f'for tumor image c   = {c}')
    plt.axis('off')

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
for i, c in enumerate(c_values, 1):
    log_image = log_transform(image_2, c)
    plt.subplot(1, len(c_values), i)
    plt.imshow(log_image, cmap='gray')
    plt.title(f'for non_tumor image c   = {c}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# 11- Unsharpen mask

In [ ]:
blured_1 = cv2.GaussianBlur(image_1,(3,3),0)
blured_2 =cv2.GaussianBlur(image_2,(3,3),0)
unsharp_1=cv2.addWeighted(image_1, 1.5, blured_1, -0.5 ,0 )
unsharp_2=cv2.addWeighted(image_2, 1.5, blured_2, -0.5, 0 )
print("this unsharpen mask used a gaussian filter with kernel 3*3")
plt.figure(figsize=(14,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title(" original tumor image ")

plt.subplot(142)
plt.imshow(unsharp_1, cmap='gray')
plt.title(" tumor unsharp mask image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")

plt.subplot(144)
plt.imshow(unsharp_2, cmap='gray')
plt.title(" non_tumor unsharp mask image ")
plt.show()


In [ ]:
blured_1 = cv2.GaussianBlur(image_1,(7,7),0)
blured_2 =cv2.GaussianBlur(image_2,(7,7),0)
unsharp_1=cv2.addWeighted(image_1, 1.5, blured_1, -0.5 ,0 )
unsharp_2=cv2.addWeighted(image_2, 1.5, blured_2, -0.5, 0 )
print("this unsharpen mask used a gaussian filter with kernel 7*7")
plt.figure(figsize=(14,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title(" original tumor image ")

plt.subplot(142)
plt.imshow(unsharp_1, cmap='gray')
plt.title(" tumor unsharp mask image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")

plt.subplot(144)
plt.imshow(unsharp_2, cmap='gray')
plt.title(" non_tumor unsharp mask image ")
plt.show()

# 11- prewitt filter

In [ ]:
prewitt_x= np.array([
    [-1,0,1],
    [-1,0,1],
    [-1,0,1]
])

prewitt_y= np.array([
    [-1,0,1],
    [-1,0,1],
    [-1,0,1]
])



In [ ]:
Gx_image_1=cv2.filter2D(image_1,-1 , prewitt_x)
Gx_image_1= cv2.filter2D(image_1 , -1, prewitt_y)
edges_image_1= cv2.max(np.abs(Gx_image_1), np.abs(Gx_image_1))

Gx_image_1=cv2.filter2D(image_2,-1 , prewitt_x)
Gy_image_2= cv2.filter2D(image_2 , -1, prewitt_y)
edges_image_2= cv2.max(np.abs(Gx_image_1), np.abs(Gy_image_2))

In [ ]:
plt.figure(figsize=(18,6))

plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(edges_image_1, cmap='gray')
plt.title(" prewitt filtered tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")

plt.subplot(144)
plt.imshow(edges_image_2, cmap='gray')
plt.title(" prewitt filtered non_tumor image ")
plt.show()

# 12- sobel filter

In [ ]:
sobel_x_image_1 = cv2.Sobel(image_1, cv2.CV_64F, 1, 0, ksize=3)   # cv2.cv-64f used to store all the gradient as it produce -ve gradient will the unit8 store only the +ve gradient
sobel_y_image_1 = cv2.Sobel(image_1, cv2.CV_64F, 0, 1, ksize=3)

sobel_x_image_2 = cv2.Sobel(image_2, cv2.CV_64F, 1, 0, ksize=3)
sobel_y_image_2 = cv2.Sobel(image_2, cv2.CV_64F, 0, 1, ksize=3)

sobel_edges_image_1 = np.sqrt(sobel_x_image_1**2 + sobel_y_image_1**2)
sobel_edges_image_1_final = np.uint8((255 * sobel_edges_image_1 / np.max(sobel_edges_image_1)))

sobel_edges_image_2 = np.sqrt(sobel_x_image_2**2 + sobel_y_image_2**2)
sobel_edges_image_2_final  = np.uint8((255 * sobel_edges_image_2 / np.max(sobel_edges_image_2)))


In [ ]:
plt.figure(figsize=(18,6))

plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(sobel_edges_image_1_final, cmap='gray')
plt.title(" sobel filtered tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original non_tumor image ")

plt.subplot(144)
plt.imshow(sobel_edges_image_2_final, cmap='gray')
plt.title(" sobel filtered non_tumor image ")
plt.show()

# 13-Frequency filters

# 13-a Low pass filter

In [ ]:
image_1.shape

In [ ]:
spectrum_1 = fft2(image_1,(630, 571))
spectrum_1 = fftshift(spectrum_1)

rows, cols = 630,571
crow, ccol = rows//2, cols//2

mask = np.zeros((rows, cols), np.uint8)
mask[crow-100:crow+100, ccol-100:ccol+100] = 1
plt.imshow(mask, cmap='gray')

In [ ]:
image_2.shape

In [ ]:
spectrum_2 = fft2(image_2,(630, 630))
spectrum_2= fftshift(spectrum_2)

rows, cols = image_2.shape
crow, ccol = rows//2, cols//2

mask = np.zeros((rows, cols), np.uint8)
mask[crow-100:crow+100, ccol-100:ccol+100] = 1
plt.imshow(mask, cmap='gray')

In [ ]:
mask.shape

In [ ]:
fshift_masked_1 = spectrum_1 * mask
img_filtered_1 = np.abs(ifft2(fshift_masked_1))

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(image_1, cmap='gray')
plt.title('Original tumor  Image')
plt.subplot(122)
plt.imshow(img_filtered_1, cmap='gray')
plt.title('low pass filtred tumor Image')
plt.show()

In [ ]:
fshift_masked_2 = spectrum_2 * mask
img_filtered_2 = np.abs(ifft2(fshift_masked_2))

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(image_2, cmap='gray')
plt.title('Original tumor  Image')
plt.subplot(122)
plt.imshow(img_filtered_2, cmap='gray')
plt.title('low pass filtred tumor Image')
plt.show()

# 13-b High pass filter

In [ ]:
spectrum_3 = fft2(image_1,(630,571))
spectrum_3= fftshift(spectrum_3)

rows, cols = 630,571
crow, ccol = rows//2, cols//2

# Create a mask for high-pass filter
mask = np.ones((rows, cols), np.uint8)
mask[crow-30:crow+30, ccol-30:ccol+30] = 0
plt.imshow(mask,cmap='gray')
plt.title(" original tumor image spectrum")


In [ ]:
spectrum_4 = fft2(image_2,(630,630))
spectrum_4= fftshift(spectrum_4)

rows, cols = image_2.shape
crow, ccol = rows//2, cols//2

# Create a mask for high-pass filter
mask = np.ones((rows, cols), np.uint8)
mask[crow-30:crow+30, ccol-30:ccol+30] = 0
plt.imshow(mask,cmap='gray')
plt.title("original non_tumor spectrum")

In [ ]:
fshift_masked_3 = spectrum_3 * mask
img_filtered_3 = np.abs(ifft2(fshift_masked_3))

plt.imshow(img_filtered_3, cmap='gray')
plt.title('Filtered tumor Image')
plt.show()

In [ ]:
fshift_masked_4 = spectrum_4 * mask
img_filtered_4= np.abs(ifft2(fshift_masked_4))

plt.imshow(img_filtered_4, cmap='gray')
plt.title('Filtered non_tumor Image')
plt.show()

# 14-Morphological operations

# 14-a Dilation operation

In [ ]:
kernel= np.ones((5,5), np.uint8)
dilated_1= cv2.dilate(image_1, kernel , iterations=1)
dilated_2=cv2.dilate(image_2, kernel , iterations=1)
plt.figure(figsize=(14,6))
plt.subplot (141)
plt.imshow(image_1, cmap='gray')
plt.title(" original tumor image ")

plt.subplot(142)
plt.imshow(dilated_1, cmap='gray')
plt.title("dilated tumor image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title(" original normal image ")

plt.subplot(144)
plt.imshow(dilated_2, cmap='gray')
plt.title(" normal dilated image ")

plt.show()

# 14--b Erossion

In [ ]:
kernel =np.ones((5,5), np.uint8)
eroed_1 = cv2.erode(image_1, kernel ,iterations =1  )
eroed_2 = cv2.erode(image_2, kernel , iterations=1)
plt.figure(figsize=(14,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(eroed_1, cmap='gray')
plt.title("tumor eroed image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")

plt.subplot(144)
plt.imshow(eroed_2, cmap='gray')
plt.title("non_tumor eroed image")
plt.show()

# 14-c Opening

In [ ]:
kernel = np.ones((5,5),np.uint8)
opening_1 = cv2.morphologyEx(image_1, cv2.MORPH_OPEN, kernel)
opening_2 = cv2.morphologyEx(image_2, cv2.MORPH_OPEN, kernel)

plt.figure(figsize=(14,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(opening_1, cmap='gray')
plt.title("tumor opened image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")

plt.subplot(144)
plt.imshow(opening_2, cmap='gray')
plt.title("non_tumor opened image")
plt.show()

# 14-d Close operation

In [ ]:
kernel = np.ones((5,5),np.uint8)
closed_1 = cv2.morphologyEx(image_1, cv2.MORPH_CLOSE, kernel)
closed_2 = cv2.morphologyEx(image_2, cv2.MORPH_CLOSE, kernel)

plt.figure(figsize=(14,6))
plt.subplot(141)
plt.imshow(image_1, cmap='gray')
plt.title("original tumor image ")

plt.subplot(142)
plt.imshow(closed_1, cmap='gray')
plt.title("tumor closed image ")

plt.subplot(143)
plt.imshow(image_2, cmap='gray')
plt.title("original non_tumor image ")

plt.subplot(144)
plt.imshow(closed_2, cmap='gray')
plt.title("non_tumor closed image")
plt.show()

# Create datafram from dataset

In [ ]:
data_path = "/kaggle/input/barin-tumor-mri-dataset"
folds = os.listdir(data_path)
data_paths = []
labels = []
for fold in folds:
    folderPath = os.path.join(data_path,fold)
    files = os.listdir(folderPath)
    for file in files:
     data_paths.append(os.path.join(folderPath,file))
     labels.append(fold.split(' ')[0])
data_df = pd.DataFrame(data = {"file_paths": data_paths, "labels": labels})

# Apply unsharpen mask to all the dataset

In [ ]:

output_path = "/kaggle/working/processed_mri_images_unsharp"
os.makedirs(output_path, exist_ok=True)

processed_paths = []
processed_labels = []

for idx, row in tqdm(data_df.iterrows(), total=len(data_df), desc="Processing images"):
    img_path = row['file_paths']
    label = row['labels']

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Warning: Could not read {img_path}")
        continue

    # -------------------------------
    # Unsharp Masking
    # -------------------------------
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    unsharp_img = cv2.addWeighted(img, 1.5, blur, -0.5, 0)

    # Create label folder
    label_folder = os.path.join(output_path, label)
    os.makedirs(label_folder, exist_ok=True)

    # Save processed image
    filename = os.path.basename(img_path)
    output_img_path = os.path.join(label_folder, filename)
    cv2.imwrite(output_img_path, unsharp_img)

    processed_paths.append(output_img_path)
    processed_labels.append(label)

# Create new dataframe
processed_df = pd.DataFrame({
    "file_paths": processed_paths,
    "labels": processed_labels
})

print(f"\nProcessed dataset size: {len(processed_df)}")
print(f"Images saved to: {output_path}")


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Original vs unsharpen masked images ', fontsize=16)

for i in range(3):
    # Original image
    orig_img = cv2.imread(data_df['file_paths'].iloc[i], cv2.IMREAD_GRAYSCALE)
    axes[0, i].imshow(orig_img, cmap='gray')
    axes[0, i].set_title(f'Original - {data_df["labels"].iloc[i]}')
    axes[0, i].axis('off')

    # Processed image
    proc_img = cv2.imread(processed_df['file_paths'].iloc[i], cv2.IMREAD_GRAYSCALE)
    axes[1, i].imshow(proc_img, cmap='gray')
    axes[1, i].set_title(f'unsharpened- {processed_df["labels"].iloc[i]}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nComparison image saved to: /kaggle/working/comparison.png")

# show thw counts in datafram

In [ ]:
def count_plot(x, title, xlabel, ylabel, width, length, order = None, rotation=True, palette='summer'):
    plt.figure(figsize=(width,length))
    sns.countplot(x = x, palette=palette, order = order)
    plt.title(title, fontsize=25)
    if rotation:
        plt.xticks(rotation = 'vertical')
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=20)
    plt.show()


In [ ]:
x = data_df['labels']
order = x.value_counts().index
count_plot(x, "Labels distribution", "labels", 'Frequency', 15,12, order=order)

# split data

In [ ]:
def split_data(data_df, target_col='labels', train_size=0.8, valid_size=0.1, test_size=0.1, random_state=42):
    tr_df, dummy = train_test_split(
        data_df,
        train_size=train_size,
        random_state=random_state,
        stratify=data_df[target_col]
    )

    valid_df, test_df = train_test_split(
        dummy,
        train_size=valid_size / (valid_size + test_size),
        random_state=random_state,
        stratify=dummy[target_col]
    )

    return tr_df, valid_df, test_df

tr_df, valid_df, test_df = split_data(data_df)

# create generators

In [ ]:
def create_generators(tr_df, valid_df, test_df,
                      x_col="file_paths", y_col="labels",
                      target_size=(224, 224), batch_size=32):
    gen = ImageDataGenerator(horizontal_flip=True, fill_mode='nearest')
    tr_gen = gen.flow_from_dataframe(
        dataframe=tr_df,
        x_col=x_col,
        y_col=y_col,
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        color_mode="rgb",
        shuffle=True
    )

    valid_gen = gen.flow_from_dataframe(
        dataframe=valid_df,
        x_col=x_col,
        y_col=y_col,
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        color_mode="rgb",
        shuffle=True
    )
    test_gen = gen.flow_from_dataframe(
        dataframe=test_df,
        x_col=x_col,
        y_col=y_col,
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        color_mode="rgb",
        shuffle=False
    )

    return tr_gen, valid_gen, test_gen

tr_gen, valid_gen, test_gen = create_generators(tr_df, valid_df, test_df)

# show some samples

In [ ]:
def show_batch_samples(generator, class_names=None, n=25):
    images, labels = next(generator)
    if class_names is None:
        class_names = list(generator.class_indices.keys())
    plt.figure(figsize=(10, 6))
    for i in range(n):
        plt.subplot(int(np.sqrt(n)), int(np.sqrt(n)), i + 1)
        image = images[i] / 255.0
        plt.imshow(image)
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

show_batch_samples(tr_gen)

# Callbacks

In [ ]:

class MyCallback(keras.callbacks.Callback):
    def __init__(self,patience, stop_patience, threshold, factor, batches, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        #self.my_model = model
        self.patience = patience
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.batches = batches
        self.epochs = epochs
        self.ask_epoch = ask_epoch
        self.ask_permission = 1

        # callback variables
        self.count = 0
        self.stop_count = 0
        self.best_epoch = 1
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.best_weights = None
        self.initial_weights = None
        self.initial_lr = None

    def on_train_begin(self, logs=None):
        self.initial_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        self.best_weights = self.model.get_weights()
        self.initial_weights = self.model.get_weights()
        self.model.optimizer.learning_rate.assign(self.initial_lr)

        msg = 'Do you want model to ask you to halt training manually [y/n]?'
        print(msg)
        ans = input('').strip().lower()
        if ans == 'y':
            self.ask_permission = 1
        else:
            self.ask_permission = 0

        msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format(
            'Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor', '% Improv', 'Duration'
        )
        print(msg)
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        stop_time = time.time()
        tr_duration = stop_time - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration % 3600) // 60
        seconds = tr_duration % 60

        msg = f'Training finished in {int(hours)}h {int(minutes)}m {seconds:.2f}s'
        print(msg)

        # Restore best weights
        self.model.set_weights(self.best_weights)

    def on_train_batch_end(self, batch, logs=None):
        acc = logs.get('accuracy', 0) * 100
        loss = logs.get('loss', 0)
        msg = f'Processing batch {batch} of {self.batches} - Accuracy: {acc:.2f}% - Loss: {loss:.5f}'
        print(msg, '\r', end='')

    def on_epoch_begin(self, epoch, logs=None):
        self.ep_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        ep_end = time.time()
        duration = ep_end - self.ep_start

        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        if self.count >= self.patience - 1:
            next_lr = lr * self.factor
            self.model.optimizer.learning_rate.assign(next_lr)
            self.count = 0
            self.stop_count += 1
        else:
            next_lr = lr
        if isinstance(self.model.optimizer, tf.keras.optimizers.Adam):  # Check if the optimizer is Adam
            self.model.optimizer.learning_rate.assign(next_lr)
            #tf.keras.backend.set_value(self.model.optimizer.learning_rate, next_lr)  # Update the learning rate
        else:
            print("Optimizer is not of type Adam.")

        acc = logs.get('accuracy')
        v_acc = logs.get('val_accuracy')
        loss = logs.get('loss')
        v_loss = logs.get('val_loss')

        monitor = ''
        pimprov = 0.0
        next_lr = lr

        if acc < self.threshold:
            monitor = 'accuracy'
            pimprov = 0.0 if epoch == 0 or self.highest_tracc == 0 else (acc - self.highest_tracc) * 100 / self.highest_tracc
            if acc > self.highest_tracc:
                self.highest_tracc = acc
                self.best_weights = self.model.get_weights()
                self.count = 0
                self.stop_count = 0
                self.best_epoch = epoch + 1
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
            else:
                if self.count >= self.patience - 1:
                    next_lr = lr * self.factor
                    self.model.optimizer.learning_rate.assign(next_lr)
                    #tf.keras.backend.set_value(self.model.optimizer.learning_rate, next_lr)
                    self.count = 0
                    self.stop_count += 1
                else:
                    self.count += 1
        else:
            monitor = 'val_loss'
            pimprov = 0.0 if epoch == 0 or self.lowest_vloss == np.inf else (self.lowest_vloss - v_loss) * 100 / self.lowest_vloss
            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                self.best_weights = self.model.get_weights()
                self.count = 0
                self.stop_count = 0
                self.best_epoch = epoch + 1
            else:
                if self.count >= self.patience - 1:
                    next_lr = lr * self.factor
                    self.model.optimizer.learning_rate.assign(next_lr)
                    #tf.keras.backend.set_value(self.model.optimizer.learning_rate, next_lr)
                    self.count = 0
                    self.stop_count += 1
                else:
                    self.count += 1
                if acc > self.highest_tracc:
                    self.highest_tracc = acc

        msg = f'{epoch+1:^8}{loss:^10.3f}{acc*100:^9.2f}{v_loss:^9.3f}{v_acc*100:^9.2f}{lr:^9.5f}{next_lr:^9.5f}{monitor:^10}{pimprov:^10.2f}{duration:^8.2f}'
        print(msg)

        if self.stop_count >= self.stop_patience:
            print(f'Training stopped at epoch {epoch+1} after {self.stop_patience} LR adjustments without improvement.')
            self.model.stop_training = True

        elif self.ask_epoch is not None and self.ask_permission and (epoch + 1) >= self.ask_epoch:
            print('Enter H to halt training or a number to continue training for that many more epochs:')
            ans = input('').strip()
            if ans.lower() == 'h':
                print(f'Training stopped manually at epoch {epoch + 1}')
                self.model.stop_training = True
            else:
                try:
                    more_epochs = int(ans)
                    self.ask_epoch += more_epochs
                    print(f'Training will continue to epoch {self.ask_epoch}')
                except:
                    print('Invalid input. Training will continue.')

# Train

In [ ]:
def train_models(input_shape=(224, 224, 3),
                              num_classes=15,
                              train_gen=None,
                              valid_gen=None,
                              epochs=20):
    models_dict = {}
    efficientnet_models = {
    #"EfficientNetB0": EfficientNetB0,
    # "EfficientNetB1": EfficientNetB1,
    # "EfficientNetB2": EfficientNetB2,
    #"EfficientNetB3": EfficientNetB3,
    # "EfficientNetB4": EfficientNetB4,
    "EfficientNetB5": EfficientNetB5,
    }
    for name in efficientnet_models:
        print(f"\nTraining {name}...\n")
        base_model = efficientnet_models[name](
            include_top=False,
            weights='imagenet',
            input_shape=input_shape,
            pooling='max'
        )

        model = Sequential([
            base_model,
            Flatten(),
            BatchNormalization(axis=-1, momentum=0.99,epsilon= 0.001),
            Dense(256, activation='relu'),
            Dropout(rate= 0.45, seed= 123),
            Dense(num_classes, activation='softmax')
        ])
        initial_lr = 0.001
        #optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)

        model.compile(optimizer=Adam(learning_rate=initial_lr),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model.summary()
        batch_size = 32
        epochs = 40
        patience = 1
        stop_patience = 3
        threshold = 0.9
        factor = 0.5
        ask_epoch = 5
        batches = int(np.ceil(len(train_gen.labels) / batch_size))

        callbacks = [MyCallback(patience= patience, stop_patience= stop_patience,
                        threshold= threshold,factor= factor, batches= batches,
                        epochs= epochs, ask_epoch= ask_epoch )]
        history = model.fit(train_gen, validation_data=valid_gen, epochs=epochs, verbose=0,
                            callbacks=callbacks, validation_steps= None, shuffle= False)
        models_dict[name] = {
            "classifier": model,
            "model_history" : history
        }
        print('-' * 20)
    return models_dict

models_dict = train_models(train_gen=tr_gen,valid_gen=valid_gen)

# Test and evaluate

In [ ]:
ts_length = len(test_df)
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size
for model_name, i in models_dict.items():
    model = i["classifier"]
    print(f"Evaluating model: {model_name}")
    train_score = model.evaluate(tr_gen, steps= test_steps, verbose= 1)
    valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
    test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

    print("Train Loss: ", train_score[0])
    print("Train Accuracy: ", train_score[1])
    print('-' * 20)
    print("Validation Loss: ", valid_score[0])
    print("Validation Accuracy: ", valid_score[1])
    print('-' * 20)
    print("Test Loss: ", test_score[0])
    print("Test Accuracy: ", test_score[1])
    print('_' * 100)

# plot loss and training of the model

In [ ]:
def plot_training(models_dict):
    '''
    This function take training model and plot history of accuracy and losses with the best epoch in both of them.
    '''
    for model_name, i in models_dict.items():
        hist = i["model_history"]
        print(f"Evaluating model: {model_name}")
    # Define needed variables
        tr_acc = hist.history['accuracy']
        tr_loss = hist.history['loss']
        val_acc = hist.history['val_accuracy']
        val_loss = hist.history['val_loss']
        index_loss = np.argmin(val_loss)
        val_lowest = val_loss[index_loss]
        index_acc = np.argmax(val_acc)
        acc_highest = val_acc[index_acc]
        Epochs = [i+1 for i in range(len(tr_acc))]
        loss_label = f'best epoch= {str(index_loss + 1)}'
        acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
        plt.figure(figsize= (20, 8))
        plt.style.use('fivethirtyeight')

        plt.subplot(1, 2, 1)
        plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
        plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
        plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
        plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
        plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        print('_' * 50)
        plt.tight_layout
        plt.show()

In [ ]:
models_dict
plot_training(models_dict)

# prediction of data #

In [ ]:
 for model_name, i in models_dict.items():
        print(f"Evaluating model: {model_name}")
        model = i["classifier"]
        i["y_pred"] = model.predict(test_gen)

# Convert the predicted probabilities to class labels
        i["y_pred_labels"] = np.argmax(i["y_pred"], axis=1)

# Get the true class labels from the test data generator
        i["y_true_labels"] = test_gen.classes

# plot confusion matrix

In [ ]:
class_names = list(test_gen.class_indices.keys())
for model_name, i in models_dict.items():
        print(f"Evaluating model: {model_name}")
        model = i["classifier"]

        # Compute the confusion matrix
        confusion_mtx = confusion_matrix(i["y_true_labels"], i["y_pred_labels"])

        # Plot the confusion matrix using Seaborn heatmap
        plt.figure(figsize=(10,8))
        sns.heatmap(confusion_mtx, cmap="Blues", annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.show()

# plot classification report

In [ ]:
for model_name, i in models_dict.items():
    print(f"Evaluating model: {model_name}")
    report = classification_report(i["y_true_labels"], i["y_pred_labels"], target_names=class_names)
    print("Classification Report: ")
    print(report)
    print('_' * 50)